In [ ]:
# import dependencies
import tensorflow as tf
import numpy as np
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
model_str='fcc_model_64r'
# load your model
model=tf.keras.models.load_model('saved_model/'+model_str)
# layer name of which the covariance matrix should be calculated
# if you created a tf.keras sequential model use model.layers[<layer index>]
layer_name=model.d2
#computes the number of parameters of the layer
layer_size=1
for i in layer_name.get_weights()[0].shape:
    layer_size*=i

#conserve the numpy datatype
layer_dtype=layer_name.get_weights()[0].dtype

# load the data set
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train=x_train/255
for i in range(x_train.shape[0]):
    x_train[i]-=np.mean(x_train[i],axis=(0,1))
    x_train[i]/=np.std(x_train[i],axis=(0,1))

# choose batch size and learning rate
learning_rate = 1e-2
batch_size = 64

#create batches
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(batch_size)

#calculate number of epochs required
n_batches=train_ds.cardinality().numpy()
epochs = 12 #min int(layer_size/n_batches)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)


170498071/170498071 [==============================] - 11s 0us/step


In [ ]:
#train function for one batch
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss=loss_fn(labels, predictions)
    loss+=sum(model.losses)
  #only the specific layer is updated, s.t. the full covariance matrix is calculated
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

weights=np.zeros((epochs*n_batches,layer_size),dtype=layer_dtype)
index_1=0
for epoch in range(epochs):
  print('epoch: '+str(epoch+1)+' of '+str(epochs))
  for images, labels in train_ds:
    train_step(images, labels)
    templ=layer_name.get_weights()[0].flatten()
    weights[index_1]=templ
    index_1+=1
    
#calculate the covariance matrix
np.save('weights/dynamic_weights_'+model_str,weights)

epoch: 1 of 12
epoch: 2 of 12
epoch: 3 of 12
epoch: 4 of 12
epoch: 5 of 12
epoch: 6 of 12
epoch: 7 of 12
epoch: 8 of 12
epoch: 9 of 12
epoch: 10 of 12
epoch: 11 of 12
epoch: 12 of 12


In [ ]:
model.save('saved_model/'+model_str)